In [17]:
def loadFile(dataPath,labelPath):
    dataList = []
    #打开文件：以二进制读模式、utf-8格式的编码方式打开
    frData = open(dataPath)
    frLabel = open(labelPath)
    dataRecord = frData.read()
    labelRecord = frLabel.read()
    # print(record)
    frData.close()
    frLabel.close()
    #按照行转换为一维表即包含各行作为元素的列表，分隔符有'\r', '\r\n', \n'
    dataRecordList = dataRecord.splitlines()
    labelRecordList=labelRecord.splitlines()
    #逐行遍历：行内字段按'\t'分隔符分隔，转换为列表
    for line in dataRecordList:
        if line.strip():
            a=line.split('\t')
            a.pop()
            a=list(map(float, a))
            dataList.append(a)

            # dataList.append(list(line.split("\t")))
    #返回转换后的矩阵
    x = np.array(dataList)
    y = np.array(list(map(float, labelRecordList)))
    return np.absolute(x),y
    # return dataList

In [18]:
import numpy as np
dataPath ="./data.txt"
labelPath = "./label.txt"
tx,ty=loadFile(dataPath,labelPath)


In [21]:
print(len(tx[0]))
print(len(tx),len(ty))

300
88851 88851


In [26]:
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers
# from sklearn.model_selection import train_test_split


inputs = keras.Input(shape=(300,))
# Embed each integer in a 128-dimensional vector
x = layers.Embedding(1000, 128)(inputs)
# Add 2 bidirectional LSTMs
x = layers.Bidirectional(layers.LSTM(64, return_sequences=True))(x)
x = layers.Bidirectional(layers.LSTM(64))(x)
# Add a classifier
outputs = layers.Dense(1, activation="sigmoid")(x)
model = keras.Model(inputs, outputs)
model.summary()





Model: "model_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_8 (InputLayer)         [(None, 300)]             0         
_________________________________________________________________
embedding_5 (Embedding)      (None, 300, 128)          128000    
_________________________________________________________________
bidirectional_13 (Bidirectio (None, 300, 128)          98816     
_________________________________________________________________
bidirectional_14 (Bidirectio (None, 128)               98816     
_________________________________________________________________
dense_6 (Dense)              (None, 1)                 129       
Total params: 325,761
Trainable params: 325,761
Non-trainable params: 0
_________________________________________________________________


In [38]:
# train_x = np.random.random((20,300))
# print(train_x.shape)
# train_y = np.random.random((20,))
# print(tx[2][7])
# print(train_y.shape)
# print(ty.shape)
# val_x = np.random.random((200,))
# val_y = np.random.random((200,))
# print(val_x)
train_x = tx[:3000]
train_y = ty[:3000]
print(len(train_x),len(train_x[0]))
test_x = tx[-1000:]
test_y = ty[-1000:]
model.compile("adam", "binary_crossentropy", metrics=["accuracy"])
model.fit(train_x, train_y, epochs=4, batch_size=10,validation_split=0.2)

3000 300
Epoch 1/4
240/240 [==============================] - 91s 359ms/step - loss: 0.4649 - accuracy: 0.8008 - val_loss: 0.7815 - val_accuracy: 0.7067
Epoch 2/4
240/240 [==============================] - 94s 390ms/step - loss: 0.3501 - accuracy: 0.8392 - val_loss: 0.8804 - val_accuracy: 0.6917
Epoch 3/4
240/240 [==============================] - 93s 389ms/step - loss: 0.2921 - accuracy: 0.8800 - val_loss: 0.9186 - val_accuracy: 0.5767
Epoch 4/4
240/240 [==============================] - 93s 388ms/step - loss: 0.2350 - accuracy: 0.9058 - val_loss: 0.9837 - val_accuracy: 0.6467


In [42]:
results = model.evaluate(train_x[:1000], train_y[:1000], batch_size=34)
print("test loss, test acc:", results)
print("Generate predictions for 3 samples")
predictions = model.predict(tx[:3])
print("prediction:", predictions)

30/30 [==============================] - 4s 119ms/step - loss: 0.1516 - accuracy: 0.9530
test loss, test acc: [0.1515839546918869, 0.953000009059906]
Generate predictions for 3 samples
prediction: [[0.01189512]
 [0.98999596]
 [0.68185556]]
